## Long running persistent conversations
Truncate messages and use summarization and persistence to enable long conversations

In [ ]:
import os
import sqlite3
from dotenv import load_dotenv
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)

class StateWithSummary(MessagesState):
    summary: str

def summarize_and_truncate(state: StateWithSummary):
    prev_summary = state.get("summary")

    instruction = (
        "You are an assistant that generates and updates memory summaries from conversations between a user and an AI assistant.\n\n"
        "Your job is to produce a concise but informative summary of the user's preferences, goals, questions, and relevant details."
    )

    if prev_summary:
        instruction += (
            "\n\nHere is the existing summary you must update:\n"
            f"{prev_summary.strip()}\n\n"
            "Update this summary to include any new facts from the following conversation messages."
        )
    else:
        instruction += "\n\nCreate a summary based on the following conversation messages."

    instruction += (
        "\n\nRules:\n"
        "- Be concise but informative.\n"
        "- Focus on key facts, user intent, or assistant advice.\n"
        "- Do NOT respond to the messages.\n"
        "- The summary will be reused in future prompts.\n"
    )
    response = llm.invoke([SystemMessage(instruction)] + state["messages"][:-2])
    summary = response.content
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {
        "summary": summary,
        "messages": delete_messages
    }

def respond_to_user(state: StateWithSummary):
    summary = state.get("summary")
    messages = state["messages"]
    if summary:
        system_message = f"""You are an AI assistant. You are continuing a conversation with a user. 
    
        Here is a summary of the previous conversation:
        
        {summary}
    
        
        Use this information when responding to the user's last message IF AND ONLY IF it is relevant. 
        Do not change the topic or mix topics unless directed to by the user. 
        Stay on topic and be precise!
        """
        messages = [SystemMessage(content=system_message)] + messages
        
    assistant_response = llm.invoke(messages)
    return {"messages": [assistant_response]}
    

def should_summarize(state: StateWithSummary) -> Literal["summarize_and_truncate", END]:
    # I want to keep at least 2 messages after summary and truncation, so trigger this once the list gets to 4 messages 
    if len(state.get("messages", [])) > 3: 
        return "summarize_and_truncate"
    return END
    

graph_builder = StateGraph(StateWithSummary)
graph_builder.add_node("summarize_and_truncate", summarize_and_truncate)
graph_builder.add_node("respond_to_user", respond_to_user)

graph_builder.add_edge(START, "respond_to_user")
graph_builder.add_conditional_edges("respond_to_user", should_summarize)
graph_builder.add_edge("summarize_and_truncate", END)

db_path = "state_db/chatbot_min_longterm_convos.db"
os.makedirs(os.path.dirname(db_path), exist_ok=True)
conn = sqlite3.connect(db_path, check_same_thread=False)
memory = SqliteSaver(conn)
graph = graph_builder.compile(checkpointer=memory)

from IPython.display import Markdown
display(Markdown("```mermaid\n" + graph.get_graph().draw_mermaid() + "\n```"))

In [ ]:
config = {"thread_id": "1"}

def stream_graph_updates(user_input: str):
    for event in graph.stream(
        {"messages": [HumanMessage(user_input)]},
        config
    ):
        for value in event.values():
            if  value["messages"][-1].content:
                print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q", "bye"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except Exception as ex:
        print("Something when wrong, I need to say goodbye!")
        break